In [1]:
### import packages

import datetime
from datetime import date
import metadataFunctions as mf
#import numpy as np
#import os
from os import path
import pandas as pd
#import re
#import subprocess
#from subprocess import Popen
#import glob


In [2]:
today = date.today()
runDate = today.strftime("%Y%m%d")

In [3]:
### Define input files
RawInputFile = 'params/rawFileSN.csv'

### Define output file names
RawOutputFile = 'params/rawFileSN_' + runDate + '.csv'

In [4]:
### Define directory pathways, pull latest asset-manangement from OOI repo
amRepo = path.dirname('/Users/joeduprey/repos/asset-management/')

### load in bulk asset records from OOI asset-management github repo
sensorList = pd.read_csv(amRepo + '/bulk/sensor_bulk_load-AssetRecord.csv')
platformList = pd.read_csv(amRepo + '/bulk/platform_bulk_load-AssetRecord.csv')
cruiseList = pd.read_csv(amRepo + '/cruise/CruiseInformation.csv')

### Load in Cabled Array Deployment sheets from OOI asset-management github repo
CabledArray = pd.Series(['CE02SHBP','CE04OSBP','CE04OSPD','CE04OSPS','RS01SBPD','RS01SBPS',
                        'RS01SLBS','RS01SUM1','RS03AXBS','RS03AXPD','RS03AXPS','RS03INT2',
                        'RS03INT1','RS01SUM2','RS03CCAL','RS03ECAL','RS03ASHS'])

deploymentSheets = []
for array in CabledArray:
    deployFilePath = amRepo + '/deployment/' + array + '_Deploy.csv' 
    deploymentSheets.append(deployFilePath)
    
df_deploy = pd.concat([pd.read_csv(f, skip_blank_lines = True, comment='#') for f in deploymentSheets], ignore_index = True)


# Deployment sheet Reference Designator key to startDate, AssetID, rawFile
df_deploy_sort = df_deploy.sort_values(by=["Reference Designator","startDateTime"],ascending=False)

RefDes_dict = {}
for i in df_deploy_sort['Reference Designator'].unique():
    RefDes_dict[i] = [{'deployDate':datetime.datetime.strptime(df_deploy_sort['startDateTime'][j], '%Y-%m-%dT%H:%M:%S'), 'deployEnd':df_deploy_sort['stopDateTime'][j], 'AssetID':df_deploy_sort['sensor.uid'][j], 'deployNum':df_deploy_sort['deploymentNumber'][j],'calFile': 'none', 'calFile_verify': 'none', 'ITM_verify': 'none', 'firstRawFile':'undef', 'rawSN':'undef','rawFile_verify':'none','imageAssetID':'undef','image_verify':'none'} for j in df_deploy_sort[df_deploy_sort['Reference Designator']==i].index]


In [5]:
run_rawFileSN = 'yes' #TODO

In [6]:
#*# identify if serial number in raw (first or data) file and if so compare with assetID map in github sensorBulk        
rawCheckSensors = ['CTD','SPK','NUT','PAR','FLOR','PREST','TMPSFA','OPTAA','ADCP','PAR']
rawCheckSensors_DP = ['ENG000000','VEL3DA105','FLCDRA103','FLNTUA103','DOSTAD105',
                      'VEL3DA103','FLCDRA102','FLNTUA102','DOSTAD104',
                      'VEL3DA303','FLCDRA302','FLNTUA302','DOSTAD304']
excludeNodes = []
#*# exclude MARUM PI sensor for now since there is no raw data in the archive
excludeSensors = ['CTDPFA110']

#*# load rawFileSN.csv
rawFileSN_list = pd.read_csv(RawInputFile)
rawKeyList = []
for key, values in RefDes_dict.items():
    for deployment in RefDes_dict[key]:
        try:
            deployment['rawSN'] = rawFileSN_list.loc[(rawFileSN_list['referenceDesignator'] == key) & (rawFileSN_list['deployYear'] == int(deployment['deployDate'].strftime('%Y'))), 'rawSerialNumber'].iloc[0]
            deployment['firstRawFile'] = rawFileSN_list.loc[(rawFileSN_list['referenceDesignator'] == key) & (rawFileSN_list['deployYear'] == int(deployment['deployDate'].strftime('%Y'))), 'rawFile'].iloc[0]
            print(rawKeyList)
        except IndexError:
            if any(sensor in key[18:27] for sensor in rawCheckSensors) or any(sensor in key[18:27] for sensor in rawCheckSensors_DP):
                if all(sensor not in key[18:27] for sensor in excludeSensors):
                    if any(node_ex in key[9:14] for node_ex in excludeNodes):
                        print("node exclusion for: " + key)
                        next
                    else:
                        print('no raw file listed for: ' + key + ' ' + deployment['deployDate'].strftime('%Y'))
                        deployment['rawSN'] = '-99999'
                        deployment['firstRawFile'] = 'none'
                        rawKeyList.append(key)
                    
#*# mine raw archive and extract serial numbers from raw files
if rawKeyList and run_rawFileSN == 'yes':
    rawSN = mf.rawFileMatchExtract({key: RefDes_dict[key] for key in set(rawKeyList)})
    with open(RawOutputFile,'w') as f:
        f.write('referenceDesignator,deployYear,rawFile,rawSerialNumber\n')
        for key,values in RefDes_dict.items():
            for deployment in RefDes_dict[key]:
                if 'undef' not in deployment['firstRawFile']: 
                    f.write("%s,%s,%s,%s\n" % (key, deployment['deployDate'].strftime('%Y'), deployment['firstRawFile'], deployment['rawSN']))
                          

no raw file listed for: RS03INT2-MJ03D-11-CTDPFB307 2024
no raw file listed for: RS03ECAL-MJ03E-12-CTDPFB306 2024
['RS03INT2-MJ03D-11-CTDPFB307', 'RS03ECAL-MJ03E-12-CTDPFB306']
['RS03INT2-MJ03D-11-CTDPFB307', 'RS03ECAL-MJ03E-12-CTDPFB306']
['RS03INT2-MJ03D-11-CTDPFB307', 'RS03ECAL-MJ03E-12-CTDPFB306']
['RS03INT2-MJ03D-11-CTDPFB307', 'RS03ECAL-MJ03E-12-CTDPFB306']
no raw file listed for: RS03CCAL-MJ03F-12-CTDPFB305 2024
['RS03INT2-MJ03D-11-CTDPFB307', 'RS03ECAL-MJ03E-12-CTDPFB306', 'RS03CCAL-MJ03F-12-CTDPFB305']
['RS03INT2-MJ03D-11-CTDPFB307', 'RS03ECAL-MJ03E-12-CTDPFB306', 'RS03CCAL-MJ03F-12-CTDPFB305']
['RS03INT2-MJ03D-11-CTDPFB307', 'RS03ECAL-MJ03E-12-CTDPFB306', 'RS03CCAL-MJ03F-12-CTDPFB305']
['RS03INT2-MJ03D-11-CTDPFB307', 'RS03ECAL-MJ03E-12-CTDPFB306', 'RS03CCAL-MJ03F-12-CTDPFB305']
['RS03INT2-MJ03D-11-CTDPFB307', 'RS03ECAL-MJ03E-12-CTDPFB306', 'RS03CCAL-MJ03F-12-CTDPFB305']
no raw file listed for: RS03AXPS-SF03A-4A-NUTNRA301 2024
['RS03INT2-MJ03D-11-CTDPFB307', 'RS03ECAL-MJ03E-12

/Users/joeduprey/miniconda3/envs/mi-racle/lib/python2.7/site-packages/mi/core/log.py:108: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  parsed = yaml.load(logconfig)
/Users/joeduprey/miniconda3/envs/mi-racle/lib/python2.7/site-packages/mi/core/instrument/wrapper.py:379: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  metadata = yaml.load(open(metadata_file))

/Users/joeduprey/repos/metadataVerification/assetManagement/metadataFunctions.py:873: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  SN = str(int(df.serial_number))


2024-10-01 11:18:59,210 INFO     mi.core.instrument.publisher Publisher: max_events: 500 publish_interval: 5
2024-10-01 11:19:03,667 INFO     mi.core.instrument.publisher Publisher: max_events: 500 publish_interval: 5
2024-10-01 11:19:03,669 INFO     mi.core.instrument.publisher Publish event: {'type': 'DRIVER_ASYNC_EVENT_STATE_CHANGE', 'value': None, 'time': 1727806743.667521}
2024-10-01 11:19:03,693 INFO     mi.core.instrument.file_publisher Writing output files...
2024-10-01 11:19:03,736 INFO     mi.core.instrument.file_publisher Done writing output files...

https://rawdata.oceanobservatories.org/files/RS03AXPS/SF03A/OPTAAD301/2023/08/OPTAAD301_10.31.3.202_2101_20230831T0740_UTC.dat
conda run -n mi-racle python2 -m mi.core.instrument.playback datalog mi.instrument.wetlabs.ac_s.ooicore.driver emptyField log:// csv:// OPTAAD301_10.31.3.202_2101_20230831T0740_UTC.dat


/Users/joeduprey/miniconda3/envs/mi-racle/lib/python2.7/site-packages/mi/core/log.py:108: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  parsed = yaml.load(logconfig)
/Users/joeduprey/miniconda3/envs/mi-racle/lib/python2.7/site-packages/mi/core/instrument/wrapper.py:379: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  metadata = yaml.load(open(metadata_file))

/Users/joeduprey/repos/metadataVerification/assetManagement/metadataFunctions.py:873: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  SN = str(int(df.serial_number))


2024-10-01 11:19:06,515 INFO     mi.core.instrument.publisher Publisher: max_events: 500 publish_interval: 5
2024-10-01 11:19:06,750 INFO     mi.core.instrument.publisher Publisher: max_events: 500 publish_interval: 5
2024-10-01 11:19:06,752 INFO     mi.core.instrument.publisher Publish event: {'type': 'DRIVER_ASYNC_EVENT_STATE_CHANGE', 'value': None, 'time': 1727806746.750749}
2024-10-01 11:19:06,778 INFO     mi.core.instrument.file_publisher Writing output files...
2024-10-01 11:19:06,815 INFO     mi.core.instrument.file_publisher Done writing output files...

https://rawdata.oceanobservatories.org/files/RS03AXPS/SF03A/OPTAAD301/2022/08/OPTAAD301_10.31.3.202_2101_20220815T0740_UTC.dat
conda run -n mi-racle python2 -m mi.core.instrument.playback datalog mi.instrument.wetlabs.ac_s.ooicore.driver emptyField log:// csv:// OPTAAD301_10.31.3.202_2101_20220815T0740_UTC.dat


...

2024-10-01 12:35:42,287 INFO     mi.core.instrument.publisher Publisher: max_events: 500 publish_interval: 5
2024-10-01 12:35:42,539 INFO     mi.core.instrument.publisher Publisher: max_events: 500 publish_interval: 5
2024-10-01 12:35:42,541 INFO     mi.core.instrument.publisher Publish event: {'type': 'DRIVER_ASYNC_EVENT_STATE_CHANGE', 'value': None, 'time': 1727811342.539705}
2024-10-01 12:35:42,567 INFO     mi.core.instrument.file_publisher Writing output files...
2024-10-01 12:35:42,594 INFO     mi.core.instrument.file_publisher Done writing output files...

none
conda run -n mi-racle python2 -m mi.core.instrument.playback datalog mi.instrument.wetlabs.ac_s.ooicore.driver emptyField log:// csv:// OPTAAD106_10.33.14.8_2101_20140925T2100_UTC.dat


/Users/joeduprey/miniconda3/envs/mi-racle/lib/python2.7/site-packages/mi/core/log.py:108: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  parsed = yaml.load(logconfig)
/Users/joeduprey/miniconda3/envs/mi-racle/lib/python2.7/site-packages/mi/core/instrument/wrapper.py:379: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  metadata = yaml.load(open(metadata_file))



2024-10-01 12:35:43,871 INFO     mi.core.instrument.publisher Publisher: max_events: 500 publish_interval: 5
2024-10-01 12:35:44,114 INFO     mi.core.instrument.publisher Publisher: max_events: 500 publish_interval: 5
2024-10-01 12:35:44,116 INFO     mi.core.instrument.publisher Publish event: {'type': 'DRIVER_ASYNC_EVENT_STATE_CHANGE', 'value': None, 'time': 1727811344.11444}
2024-10-01 12:35:44,146 INFO     mi.core.instrument.file_publisher Writing output files...
2024-10-01 12:35:44,146 INFO     mi.core.instrument.file_publisher Done writing output files...

no file created
conda run -n mi-racle python2 -m mi.core.instrument.playback datalog mi.instrument.wetlabs.ac_s.ooicore.driver emptyField log:// csv:// OPTAAD106_10.33.14.8_2101_20140925T2300_UTC.dat


/Users/joeduprey/miniconda3/envs/mi-racle/lib/python2.7/site-packages/mi/core/log.py:108: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  parsed = yaml.load(logconfig)
/Users/joeduprey/miniconda3/envs/mi-racle/lib/python2.7/site-packages/mi/core/instrument/wrapper.py:379: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  metadata = yaml.load(open(metadata_file))



2024-10-01 12:35:44,835 INFO     mi.core.instrument.publisher Publisher: max_events: 500 publish_interval: 5
2024-10-01 12:35:45,074 INFO     mi.core.instrument.publisher Publisher: max_events: 500 publish_interval: 5
2024-10-01 12:35:45,077 INFO     mi.core.instrument.publisher Publish event: {'type': 'DRIVER_ASYNC_EVENT_STATE_CHANGE', 'value': None, 'time': 1727811345.075051}
2024-10-01 12:35:45,107 INFO     mi.core.instrument.file_publisher Writing output files...
2024-10-01 12:35:45,107 INFO     mi.core.instrument.file_publisher Done writing output files...

no file created
creating file list for: RS03AXPD-DP03A-01-CTDPFL304
none
https://rawdata.oceanobservatories.org/files/RS03AXPD/PD03A/eng_data/2023/09/13/sernums_hi-res_28_20230913T120040_UTC.txt
https://rawdata.oceanobservatories.org/files/RS03AXPD/PD03A/eng_data/2022/08/29/sernums_hi-res_235_20220829T030509_UTC.txt
https://rawdata.oceanobservatories.org/files/RS03AXPD/PD03A/eng_data/2021/08/27/sernums_hi-res_580_20210827T16112